In [53]:
import pandas as pd
import numpy as np
import math

# Collaborative labeling
- Generate collaborative labeling master spreadsheet
- After labeling is done, label the "stages" spreadsheet

In [55]:
df = pd.read_csv('./all_cleaned.csv')
df = df[['video_id','channel_id','rank','component','watch_video_id','bot_name','phase','homepage_level','videopage_level','phase_level','topic','strategy','note']]
df = df[(df['topic']!='random')&(df['rank']<10)]
df.rename(columns={'stain':'on_list'},inplace=True)
df.head()

,video_id,channel_id,rank,component,watch_video_id,bot_name,phase,homepage_level,videopage_level,phase_level,topic,strategy,note
0,PEWFu0aZroQ,UCXIJgqnII2ZOINSWNOGFThA,0,videopage,ya2T1XGYTR8,alt-right_delete_0,videopage_experiment,0,0,0,alt-right,delete,0
1,XmiG4KzZ4sg,UCPKAKrjoMz7POptCloy7AIQ,1,videopage,ya2T1XGYTR8,alt-right_delete_0,videopage_experiment,0,0,0,alt-right,delete,0
2,Lh0b00vL1nI,UCmRZTrJYqOMzP5J0r0snxYg,2,videopage,ya2T1XGYTR8,alt-right_delete_0,videopage_experiment,0,0,0,alt-right,delete,0
3,k6ucIShHW7Q,UCJquYOG5EL82sKTfH9aMA9Q,3,videopage,ya2T1XGYTR8,alt-right_delete_0,videopage_experiment,0,0,0,alt-right,delete,0
4,0GCDhADokLY,UC2ggrxKhqgTRx7We9vjQRCQ,4,videopage,ya2T1XGYTR8,alt-right_delete_0,videopage_experiment,0,0,0,alt-right,delete,0


In [56]:
df_homepage = df[
    (df['component']=='homepage')&
    (df['homepage_level'].isin([1,39,79]))
]

In [57]:
df_videopage = df[(df['phase']=='videopage_experiment')]
df_videopage.head()

,video_id,channel_id,rank,component,watch_video_id,bot_name,phase,homepage_level,videopage_level,phase_level,topic,strategy,note
0,PEWFu0aZroQ,UCXIJgqnII2ZOINSWNOGFThA,0,videopage,ya2T1XGYTR8,alt-right_delete_0,videopage_experiment,0,0,0,alt-right,delete,0
1,XmiG4KzZ4sg,UCPKAKrjoMz7POptCloy7AIQ,1,videopage,ya2T1XGYTR8,alt-right_delete_0,videopage_experiment,0,0,0,alt-right,delete,0
2,Lh0b00vL1nI,UCmRZTrJYqOMzP5J0r0snxYg,2,videopage,ya2T1XGYTR8,alt-right_delete_0,videopage_experiment,0,0,0,alt-right,delete,0
3,k6ucIShHW7Q,UCJquYOG5EL82sKTfH9aMA9Q,3,videopage,ya2T1XGYTR8,alt-right_delete_0,videopage_experiment,0,0,0,alt-right,delete,0
4,0GCDhADokLY,UC2ggrxKhqgTRx7We9vjQRCQ,4,videopage,ya2T1XGYTR8,alt-right_delete_0,videopage_experiment,0,0,0,alt-right,delete,0


In [58]:
df_homepage['exp_stage'] = df_homepage['homepage_level'].replace({1:1,39:2,79:3})
df_videopage['exp_stage'] = df_videopage['phase_level'].replace({0:1,1:2,2:3})

/var/folders/t6/srtzy2f51db8mxxv7g_63f6h0000gr/T/ipykernel_57921/3502909624.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_homepage['exp_stage'] = df_homepage['homepage_level'].replace({1:1,39:2,79:3})


In [60]:
df_all = pd.concat([df_homepage,df_videopage])

In [61]:
df_all['exp_stage'].value_counts()

2    2780
3    2780
1    2770
Name: exp_stage, dtype: int64

In [31]:
label_sheet = pd.get_dummies(df_all[['channel_id','topic']].set_index('channel_id')).groupby('channel_id').agg('sum').apply(lambda x: x.apply(lambda y: 1 if y>0 else 0)).replace({0: pd.NA, 1: '?'})
label_sheet.index = label_sheet.reset_index()['channel_id'].apply(lambda x: f'https://www.youtube.com/channel/{x}')
label_sheet



RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
#label_sheet.to_csv('./collab_labeling/master_collab_template.csv')

In [ ]:
topic_channel = df_all['channel_id'] + df_all['topic']
len(topic_channel.unique())

In [68]:
# Coming back after finished with manual labeling... Generate labeled stages dataset

In [72]:
labeled_df = pd.read_csv('./collab_labeling/master_collab_finished.csv')
labeled_df.drop('note',axis=1,inplace=True)
labeled_df['channel_id'] = labeled_df['channel_id'].apply(lambda x: x[len('https://www.youtube.com/channel/'):])
labeled_df = labeled_df.set_index('channel_id').stack()
#labeled_df.index = labeled_df.index.set_names(['channel_id','topic'])
labeled_df.rename('on_topic',inplace=True)
labeled_df

channel_id                              
UC-2MJlKSq9_pYk5-bdvMhnw  politics-left     False
                          politics-right    False
UC-RxXi2Xws6Uk22vp-sLbGA  politics-right    False
UC-SJ6nODDmufqBzPBwCvYvQ  politics-right    False
UC-tE6zWn-8KB73fmtLc-6zw  politics-right     True
                                            ...  
UCzqCujRKPcFwHzWOfxMHoWQ  politics-left     False
UCzsKXhHO9mU2OktYG4CspVA  politics-right    False
UCzsfa_xBwTsUGFnMPU-W6Cg  politics-right    False
UCzwAr7ne4CzL-DHpPcVN-PA  alt-right         False
UCzwFZOhCfCZTgLruMz_RWCw  alt-right         False
Name: on_topic, Length: 2620, dtype: object

In [73]:
#df_all.set_index(['channel_id','topic'],inplace=True)
df_all

,video_id,channel_id,rank,component,watch_video_id,bot_name,phase,homepage_level,videopage_level,phase_level,topic,strategy,note,exp_stage
60,ohjsypZfFVg,UCkY8CvV8WQFe87CZGmvuYHA,0,homepage,NaN,alt-right_delete_0,stain,1,2,1,alt-right,delete,0,1
61,3zlvqFcinr8,UCXHcH4CggCrvIpm0KM0kHQg,1,homepage,NaN,alt-right_delete_0,stain,1,2,1,alt-right,delete,0,1
62,9Xd8xq06FCw,UCCND6a0H56zHL4YuY226ZOQ,2,homepage,NaN,alt-right_delete_0,stain,1,2,1,alt-right,delete,0,1
63,sLTvQnjEkRU,UCAaZm4GcWqDg8358LIx3kmw,3,homepage,NaN,alt-right_delete_0,stain,1,2,1,alt-right,delete,0,1
64,PmNyNJFCugo,UCvrmr8Nfd86dG7pnSHC7c6Q,4,homepage,NaN,alt-right_delete_0,stain,1,2,1,alt-right,delete,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407242,XojPKzDyZ30,UCUUWDzrkuEztzddaW_sgQbA,5,videopage,WS8UmDc6UD4,politics-right_watch_4,videopage_experiment,80,74,2,politics-right,watch,4,3
407243,_MkOw1URMJE,UCXIJgqnII2ZOINSWNOGFThA,6,videopage,WS8UmDc6UD4,politics-right_watch_4,videopage_experiment,80,74,2,politics-right,watch,4,3
407244,23CnNytmsRU,UC5jpAHg6Smr6Hpout2JeLvA,7,videopage,WS8UmDc6UD4,politics-right_watch_4,videopage_experiment,80,74,2,politics-right,watch,4,3
407245,TV5ubJq39h0,UCO0akufu9MOzyz3nvGIXAAw,8,videopage,WS8UmDc6UD4,politics-right_watch_4,videopage_experiment,80,74,2,politics-right,watch,4,3


In [74]:
df_all_labeled = df_all.join(labeled_df, on=['channel_id','topic'])#.tail(50)

In [75]:
df_all_labeled['on_topic'].value_counts()

False    6516
True     1765
Name: on_topic, dtype: int64

In [77]:
df_all_labeled.to_csv('./all_cleaned_labeled.csv')